# CTE

Gunakan rexon_metals.db!

In [1]:
# Pertama kita perlu untuk setup drive di google colab agar kita dapat mengakses file yang sudah kita simpan dalam
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#import packages
import os
import sqlite3
import pandas as pd

In [3]:
# Kemudian kita definisikan path folder dimana file database kita disimpan
path = "gdrive/MyDrive/Folder Saya/Narasio Data - Data Analyst/Week 4 - SQL for Data Analysis/Dataset - SQL for Data Analysis/"

In [4]:
# untuk mengecek ada file apa saja didalam path yang digunakan
os.listdir(path)

['rexon_metals.db',
 'weather_stations.db',
 'transportation_sales.db',
 'transportasion_sales.db']

In [5]:
# Kemudian kembali membuat connection ke database yang udah ada
conn = sqlite3.connect(path + 'rexon_metals.db')

# Membuat cursor object untuk memanggil SQL statement
cur = conn.cursor()

In [6]:
# Kemudian kita dapat execute SQL statement. Kali ini kita akan mengecek terdapat tabel apa saja di rexon_metals.db
cur.execute("""
              SELECT 
                  name 
              FROM sqlite_master 
              WHERE type='table'
            """)

print(cur.fetchall())

[('PRODUCT',), ('sqlite_sequence',), ('CUSTOMER',), ('CUSTOMER_ORDER',), ('stocks',)]


In [9]:
pd.read_sql_query("""
                  SELECT 
                      * 
                  FROM
                    PRODUCT
                  """, conn)

,PRODUCT_ID,DESCRIPTION,PRICE
0,1,Copper,7.51
1,2,Aluminum,2.58
2,3,Silver,15.00
3,4,Steel,12.31
4,5,Bronze,4.00
5,6,Duralumin,7.60
6,7,Solder,14.16
7,8,Stellite,13.31
8,9,Brass,4.75


In [10]:
pd.read_sql_query("""
                  SELECT 
                      * 
                  FROM
                    CUSTOMER
                  """, conn)

,CUSTOMER_ID,NAME,REGION,STREET_ADDRESS,CITY,STATE,ZIP
0,1,LITE Industrial,Southwest,729 Ravine Way,Irving,TX,75014
1,2,Rex Tooling Inc,Southwest,6129 Collie Blvd,Dallas,TX,75201
2,3,Re-Barre Construction,Southwest,9043 Windy Dr,Irving,TX,75032
3,4,Prairie Construction,Southwest,264 Long Rd,Moore,OK,62104
4,5,Marsh Lane Metal Works,Southeast,9143 Marsh Ln,Avondale,LA,79782


In [11]:
pd.read_sql_query("""
                  SELECT 
                      * 
                  FROM
                    CUSTOMER_ORDER
                  """, conn)

,ORDER_ID,ORDER_DATE,SHIP_DATE,CUSTOMER_ID,PRODUCT_ID,ORDER_QTY,SHIPPED
0,1,2015-05-15,2015-05-18,1,1,450,false
1,2,2015-05-18,2015-05-21,3,2,600,false
2,3,2015-05-20,2015-05-23,3,5,300,false
3,4,2015-05-18,2015-05-22,5,4,375,false
4,5,2015-05-17,2015-05-20,3,2,500,false


### Total hasil penjualan setiap produk

In [21]:
# Step 1: mengetahui total kuantiti terjual
pd.read_sql_query("""
                  SELECT 
                    SUM(CUSTOMER_ORDER.ORDER_QTY) AS Total_Kuantiti,
                    CUSTOMER_ORDER.PRODUCT_ID,
                    PRODUCT.DESCRIPTION 
                  FROM
                    CUSTOMER_ORDER
                  INNER JOIN
                    PRODUCT
                  ON CUSTOMER_ORDER.PRODUCT_ID = PRODUCT.PRODUCT_ID
                  GROUP BY
                    2, 3
                  """, conn)

,Total_Kuantiti,PRODUCT_ID,DESCRIPTION
0,450,1,Copper
1,1100,2,Aluminum
2,375,4,Steel
3,300,5,Bronze


In [19]:
# Step 2: mengalikan kuantiti terjual dengan harga jual
pd.read_sql_query("""
                  SELECT 
                    (CUSTOMER_ORDER.ORDER_QTY * PRODUCT.PRICE) AS Total,
                    PRODUCT.PRODUCT_ID,
                    PRODUCT.DESCRIPTION
                  FROM
                    CUSTOMER_ORDER
                  INNER JOIN
                    PRODUCT
                  ON CUSTOMER_ORDER.PRODUCT_ID = PRODUCT.PRODUCT_ID
                  GROUP BY
                    2, 3
                  """, conn)

,Total,PRODUCT_ID,DESCRIPTION
0,3379.50,1,Copper
1,1548.00,2,Aluminum
2,4616.25,4,Steel
3,1200.00,5,Bronze
